# Tutorial: HDF5 headers
This tutorial will show you how to estimate the shape and sizes of beams from the LOFAR telescope.

First, let's load the python libraries that we will use.

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import h5py
import glob
%pylab inline

HDF5 files store both data as well as the associated metadata in groups. The image below gives a overview of the HDF5 file structure.
![title](img/hdf5_structure4.jpg)

Beamformed LOFAR data uses this structure, where the formatting of the HDF5 file name in sub-array pointing, beam, and Stokes parameter, is represented in the file.

The following commands show the group structure of a LOFAR HDF5 file.

In [ ]:
h5 = h5py.File("L631533/L631533_SAP000_B000_S0_P000_bf.h5", "r")

In [ ]:
def print_name(name):
    print(name)

In [ ]:
h5.visit(print_name)

You will recognize the `SUB_ARRAY_POINTING_000/BEAM_000/STOKES_0` grouping. Each of these groups and subgroups has a list of *attributes*, which hold metadata. The following command shows the attributes of the `root` ("/") group.

In [ ]:
group = h5["/"]
keys = sorted(["%s"%item for item in sorted(list(group.attrs))])
for key in keys:
    print(key + " = " + str(group.attrs[key]))

## Exercise 1
Have a look at the attributes of the other subgroups in the file. In particular try to find the attributes which describe;
1. The number of sub-array pointings in this observation,
2. The number of tied-array beams within a sub-array,
3. The total bandwidth,
4. The integration time,
5. Which Stokes parameter is contained in this file.

Use the `.attrs["<keyword>"]` function to obtain the values of these parameters.

The sub-array pointing group provides the pointing right ascension and declination as `POINT_RA` and `POINT_DEC`, as well as keywords describing their units. Let's first the pointing right ascension and declination of all the sub-array pointings in this observation.

We can use `glob` to create a list with HDF5 headers of the first beam (`B000`) of each sub-array pointing.

In [ ]:
sapfnames = sorted(glob.glob("L631533/L631533_SAP???_B000_S0_P000_bf.h5"))

The following piece of code loops over these files to obtain the sub-array pointing identifiers, and then obtains the pointing right ascension and declination of each.

In [ ]:
sap_ra = []
sap_dec = []
for sapfname in sapfnames:
    h5 = h5py.File(sapfname, "r")
    sap_ids = [key for key in h5.keys() if "SUB_ARRAY_POINTING" in key]
    for sap_id in sap_ids:
        sap_ra.append(h5[sap_id].attrs['POINT_RA'])
        sap_dec.append(h5[sap_id].attrs['POINT_DEC'])

We can now plot these pointings.

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(sap_ra, sap_dec, 'o')
plt.xlabel("Right ascension (deg)")
plt.ylabel("Declination (deg)")
plt.title("L631533 sub-array pointings")

## Exercise 2
Use the code from the previous cells as an example to obtain the pointing right ascension and declination of each beam in each of the three sub-array pointings. Make a plot of all the beam pointings and sub-array pointings.

## Exercise 3
You will have noticed the `COORDINATES` sub-group of the beam group. Investigate this group and the groups contained within them.
1. What are the dimensions and units of the two coordinate axes?
2. Compare the setup of the two coordinate axis. How does their setup differ?
3. Extract the frequencies of the channels and show that these match the channel width defined in the beam sub-group.